In [1]:
import numpy as np
import pandas as pd

In [ ]:
base_sql = """
WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 需要满足回溯日期
        -- back_date要求YYYYMMDD格式
        select 
            mobile_md5,
            substr(back_date, 0, 10) as back_date
        from risk.loan_intent_samples_20241023
    )smp
    group by mobile_md5, back_date
),
customer_dwb_sms AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_type, 
        ind_tag
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_loan.sign,
            dwb_loan.event_type,
            case when dwb_loan.sign regexp "抖音月付|放心借|微众银行|花呗" then 'indA'
                 when dwb_loan.sign regexp "美团月付|携程金融|360借|拍拍贷|分期乐|拍拍金融|360贷款|上海拍拍贷|美团金融服务" then 'indB'
                 when dwb_loan.sign regexp "还呗|信用飞钱包|省呗|乐享借|融360|洋钱罐借款|众安金融|分期金融|信用飞|乐逸花|极融借款|好分期|信用卡贷|金瀛分期|小赢卡贷|宜享花|众安贷借钱|融逸花|卡贷金融|吉用花|时光分期|小花钱包|借钱呗|金豆花|信用飞金融|来分期|小花借款|好会借|乐贷分期|及贷|榕树贷款|融易分期|借小花|薇钱包|你我贷|玖富借条" then 'indC'
                 when dwb_loan.sign regexp "金融调解|数科金融|满松科技|利信普惠|普信金融|卡卡金融|和信普惠|和信金融|利信金融|数信普惠|普惠快信|普惠信息|首山金融|普惠金融|数信普惠|玖富万卡|钱站|鹰潭市金融纠纷调解中心|国美易卡|普惠分期|上海金融|数科纠纷调解中心" or sign in ('玖富') then 'indD'
                 when dwb_loan.sign regexp "消费分期|消费金融|招联金融|移动白条|捷信金融|马上金融|中原消费金融|马上消费|分期消费" then 'indE'
                 when dwb_loan.sign regexp "银行|农信|农商" then 'indF'
            else 'indG' end as ind_tag
        from cus_sample cus_smp
        left join(
            select
                phone,
                sign,
                event_type,
                from_unixtime(unix_timestamp(the_date, 'yyyyMMdd'), 'yyyy-MM-dd') as the_date
            from sms_bd_data.sms_dwb_fin_loan_event_fdt
        )dwb_loan
        on cus_smp.mobile_md5=dwb_loan.phone 
        where cus_smp.back_date > dwb_loan.the_date and dwb_loan.the_date >= date_sub(cus_smp.back_date, 360)
    )cus_dwb
    group by mobile_md5, back_date, sign, event_type, ind_tag
)
"""

In [ ]:
# win_days = [15, 30, 90, 180, 360]
# ind_arr = ["indA", "indB", "indC", "indD", "indE", "indF", "indG"]
# event_arr = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B09", "B10"]
# # 金融事件总次数
# total_times = ""
# for win in win_days:
#     total_times += f"""
#         ,SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, {win}) THEN 1 ELSE 0 END) AS sp_fin_loan_event_total_times_{win}d
#         ,COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, {win}) then sign ELSE NULL END) AS sp_fin_loan_plat_cnt_{win}d
#     """
# # 不同行业行为次数
# for win in win_days:
#     for ind in ind_arr:
#         total_times += f"""
#         ,SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, {win}) AND ind_tag='{ind}' THEN 1 ELSE 0 END) AS sp_fin_loan_event_{ind.lower()}_times_{win}d
#         ,COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, {win}) AND ind_tag='{ind}' then sign ELSE NULL END) AS sp_fin_loan_{ind.lower()}_plat_cnt_{win}d
#         """

# # 不同事件类型行为次数
# for win in win_days:
#     for event in event_arr:
#         total_times += f"""
#         ,SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, {win}) AND event_type='{event}' THEN 1 ELSE 0 END) AS sp_fin_loan_event_{event.lower()}_times_{win}d
#         ,COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, {win}) AND event_type='{event}' then sign ELSE NULL END) AS sp_fin_loan_event_{event.lower()}_plat_cnt_{win}d
#         """

# # 不同行业与事件类型的组合次数
# for win in win_days:
#     for ind in ind_arr:
#         for event in event_arr:
#             total_times += f"""
#             ,SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, {win}) AND ind_tag='{ind}' AND event_type='{event}' THEN 1 ELSE 0 END) AS sp_fin_loan_{ind.lower()}_event_{event.lower()}_times_{win}d
#             ,COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, {win}) AND ind_tag='{ind}' AND event_type='{event}' then sign ELSE NULL END) AS sp_fin_loan_{ind.lower()}_event_{event.lower()}_plat_cnt_{win}d
#             """ 

# aggregated_sql = base_sql + f"""
# ,aggregated_data AS (
#     SELECT
#         mobile_md5
#         ,back_date
#         {total_times}
#     FROM customer_dwb_sms
#     GROUP BY mobile_id, back_date
# )
# """
# print(aggregated_sql)



WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 需要满足回溯日期
        -- back_date要求YYYYMMDD格式
        select 
            mobile_md5,
            substr(back_date, 0, 10) as back_date
        from risk.loan_intent_samples_20241023
    )smp
    group by mobile_md5, back_date
),
customer_dws_logs AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_type, 
        ind_tag
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_loan.sign,
            dwb_loan.event_type,
            case when dwb_loan.sign regexp "抖音月付|放心借|微众银行|花呗" then 'indA'
                 when dwb_loan.sign regexp "美团月付|携程金融|360借|拍拍贷|分期乐|拍拍金融|360贷款|上海拍拍贷|美团金融服务" then 'indB'
                 when dwb_loan.sign regexp "还呗|信用飞钱包|省呗|乐享借|融360|洋钱罐借款|众安金融|分期金融|信用飞|乐逸花|极融借款|好分期|信用卡贷|金瀛分期|小赢卡贷|宜享花|众安贷借钱|融逸花|卡贷金融|吉用花|时光分期|小花钱包|借钱呗|金豆花|信用飞金融|来分期|小花借款|好会借|乐贷分期|及贷|榕树贷款|融易分期|借小花|薇钱包|你我贷|玖富借条" then 'indC'
       

In [ ]:
def generate_fin_loan_stat_fields(win_days, ind_arr, event_arr):
    """
    封装统计字段生成逻辑，返回格式化的字段字符串
    :param win_days: 时间窗口列表
    :param ind_arr: 行业标签列表
    :param event_arr: 事件类型列表
    :return: 拼接好的统计字段字符串
    """
    fields = []
    # 统一时间条件模板（减少重复书写）
    time_cond_template = "{time_col} < back_date AND {time_col} >= date_sub(back_date, {win})"
    
    # 1. 金融事件总次数
    for win in win_days:
        # 时间条件（event_time 用于SUM，the_date 用于COUNT）
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="event_time")
        
        # 总次数字段
        fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} THEN 1 ELSE 0 END) AS sp_fin_loan_event_total_times_{win}d""")
        # 平台数字段
        fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} THEN sign ELSE NULL END) AS sp_fin_loan_plat_cnt_{win}d""")

    # 2. 不同行业行为次数
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="the_date")
        for ind in ind_arr:
            ind_lower = ind.lower()
            # 行业次数字段
            fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} AND ind_tag='{ind}' THEN 1 ELSE 0 END) AS sp_fin_loan_{ind_lower}_times_{win}d""")
            # 行业平台数字段
            fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} AND ind_tag='{ind}' THEN sign ELSE NULL END) AS sp_fin_loan_{ind_lower}_plat_cnt_{win}d""")

    # 3. 不同事件类型行为次数
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="the_date")
        for event in event_arr:
            event_lower = event.lower()
            # 事件次数字段
            fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} AND event_type='{event}' THEN 1 ELSE 0 END) AS sp_fin_loan_event_{event_lower}_times_{win}d""")
            # 事件平台数字段
            fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} AND event_type='{event}' THEN sign ELSE NULL END) AS sp_fin_loan_event_{event_lower}_plat_cnt_{win}d""")

    # 4. 行业+事件类型组合次数
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="the_date")
        for ind in ind_arr:
            ind_lower = ind.lower()
            if ind_lower == "indd":
                continue
            for event in event_arr:
                event_lower = event.lower()
                # 组合次数字段
                fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} AND ind_tag='{ind}' AND event_type='{event}' THEN 1 ELSE 0 END) AS sp_fin_loan_{ind_lower}_event_{event_lower}_times_{win}d""")
                # 组合平台数字段
                fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} AND ind_tag='{ind}' AND event_type='{event}' THEN sign ELSE NULL END) AS sp_fin_loan_{ind_lower}_event_{event_lower}_plat_cnt_{win}d""")

    # 5. 不同事件行为最近一次与最远一次距今天数
    time_cond = time_cond_template.format(time_col="event_time", win=win)
    for event in event_arr:
        event_lower = event.lower()
        # 单个事件最近天数
        fields.append(f"""
        datediff(back_date, MAX(CASE WHEN {time_cond} AND event_type='{event}' THEN event_time ELSE NULL END)) AS sp_fin_loan_event_{event_lower}_recent_dura_d_beforep_{win}d""")
        # 单个事件最远天数
        fields.append(f"""
        datediff(back_date, MIN(CASE WHEN {time_cond} AND event_type='{event}' THEN event_time ELSE NULL END)) AS sp_fin_loan_event_{event_lower}_remote_dura_d_beforep_{win}d""") 
    
    for ind in ind_arr:
        ind_lower = ind.lower()
        for event in event_arr:
            event_lower = event.lower()
            # 组合事件最近天数
            fields.append(f"""
            datediff(back_date, MAX(CASE WHEN {time_cond} AND ind_tag='{ind}' AND event_type='{event}' THEN event_time ELSE NULL END)) AS sp_fin_loan_{ind_lower}_event_{event_lower}_recent_dura_d_beforep_{win}d""")
            # 组合事件最远天数
            fields.append(f"""
            datediff(back_date, MIN(CASE WHEN {time_cond} AND ind_tag='{ind}' AND event_type='{event}' THEN event_time ELSE NULL END)) AS sp_fin_loan_{ind_lower}_event_{event_lower}_remote_dura_d_beforep_{win}d""")
    
        # 拼接所有字段，去除多余换行和空格，保持格式整洁
    return ",\n".join([f.strip() for f in fields])

def generate_fin_loan_stat_fields_levelv2(win_days, event_arr, ind_arr):
    fields = []
    
    # 6. 同一事件不同时间窗口占
    # 6. 不同事件与不同行业占总事件比率
    for win in win_days:
        # 6.1 单个事件占总事件的比率
        for event in event_arr:
            event_lower = event.lower()
            fields.append(f"""
            ROUND(CASE WHEN sp_fin_loan_total_times_{win}d = 0 THEN 0 ELSE sp_fin_loan_event_{event_lower}_times_{win}d / sp_fin_loan_total_times_{win}d END, 6) AS sp_fin_loan_event_{event_lower}_ratio_total_{win}d""")
        
        # 6.2 单个行业占总事件的比率
        for ind in ind_arr:
            ind_lower = ind.lower()
            fields.append(f"""
            ROUND(CASE WHEN sp_fin_loan_total_times_{win}d = 0 THEN 0 ELSE sp_fin_loan_{ind_lower}_times_{win}d / sp_fin_loan_total_times_{win}d END, 6) AS sp_fin_loan_{ind_lower}_ratio_total_{win}d""")

    # 先定义窗口映射（小窗口→大窗口，可根据业务调整）
    win_mapping = {
        15: 30,   # 15天/30天
        30: 90,   # 30天/90天
        60: 360   # 60天/360天
    }
    # 仅遍历指定的小窗口
    for small_win in [15, 30, 60]:
        big_win = win_mapping[small_win]
        
        # 7.1 纯事件维度：单个事件小窗口次数 / 该事件大窗口次数（保留6位小数）
        for event in event_arr:
            event_lower = event.lower()
            fields.append(f"""
            ROUND(CASE WHEN sp_fin_loan_event_{event_lower}_times_{big_win}d = 0 THEN 0 ELSE sp_fin_loan_event_{event_lower}_times_{small_win}d / sp_fin_loan_event_{event_lower}_times_{big_win}d END, 6) AS sp_fin_loan_event_{event_lower}_ratio_{small_win}to{big_win}d""")
        
        # 7.2 行业+事件组合维度：某行业下某事件小窗口次数 / 该行业下该事件大窗口次数（保留2位小数）
        for ind in ind_arr:
            ind_lower = ind.lower()
            if ind_lower == "indd":
                continue
            for event in event_arr:
                event_lower = event.lower()
                fields.append(f"""
                ROUND(CASE WHEN sp_fin_loan_{ind_lower}_event_{event_lower}_times_{big_win}d = 0 THEN 0 ELSE sp_fin_loan_{ind_lower}_event_{event_lower}_times_{small_win}d / sp_fin_loan_{ind_lower}_event_{event_lower}_times_{big_win}d END, 6) AS sp_fin_loan_{ind_lower}_event_{event_lower}_ratio_{small_win}to{big_win}d""")
    return ",\n".join([f.strip() for f in fields])

# ===================== 配置项（抽离常量，便于修改）=====================
WIN_DAYS = [15, 30, 60, 90, 180, 360]
IND_ARR = ["indA", "indB", "indC", "indD", "indE", "indF", "indG"]
EVENT_ARR = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B09", "B10"]

# ===================== 生成最终SQL =====================
# 生成统计字段
stat_fields = generate_fin_loan_stat_fields(WIN_DAYS, IND_ARR, EVENT_ARR)

stat_fields_levelv2 = generate_fin_loan_stat_fields_levelv2(WIN_DAYS, EVENT_ARR, IND_ARR)

# 拼接最终SQL（格式对齐，便于阅读和调试）
aggregated_sql = f"""{base_sql},aggregated_data AS (
    SELECT
        mobile_md5,
        back_date,
        {stat_fields}
    FROM customer_dwb_sms
    GROUP BY mobile_md5, back_date
)
INSERT OVERWRITE TABLE sms_bd_data.customer_test_offline_sp_feature_2026010901
SELECT
   *,
    {stat_fields_levelv2}
FROM aggregated_data
"""

# 打印结果（可选：格式化输出，更易读）
print(aggregated_sql)

WITH customer_dwb_express AS (SELECT * FROM sms_bd_data.customer_dwb_express) ,aggregated_data AS (
    SELECT
        mobile_md5,
        back_date,
        SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, 15) THEN 1 ELSE 0 END) AS sp_fin_loan_event_total_times_15d,
COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, 15) THEN sign ELSE NULL END) AS sp_fin_loan_plat_cnt_15d,
SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, 30) THEN 1 ELSE 0 END) AS sp_fin_loan_event_total_times_30d,
COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, 30) THEN sign ELSE NULL END) AS sp_fin_loan_plat_cnt_30d,
SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, 90) THEN 1 ELSE 0 END) AS sp_fin_loan_event_total_times_90d,
COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, 90) THEN sign ELSE NULL END) AS sp_fin_loan_plat_cnt_90d,
SUM(CASE WHEN 

In [28]:
def generate_express_stat_fields(win_days, event_arr):
    """
    封装快递（express）模块特征生成逻辑（仅事件维度，无行业分组）
    :param win_days: 时间窗口列表
    :param event_arr: 事件类型列表
    :return: 拼接好的统计字段字符串
    """
    fields = []
    # 统一时间条件模板（单大括号适配Python format）
    time_cond_template = "{time_col} < back_date AND {time_col} >= date_sub(back_date, {win})"
    
    # ========== 1. 快递事件总次数 + 平台数 ==========
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="the_date")
        
        # 总次数字段
        fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} THEN 1 ELSE 0 END) AS sp_express_event_total_times_{win}d""")
        # 平台数字段（保留原逻辑，无行业分组）
        fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} THEN sign ELSE NULL END) AS sp_express_plat_cnt_{win}d""")

    # ========== 2. 不同事件类型行为次数 + 平台数 ==========
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="the_date")
        for event in event_arr:
            event_lower = event.lower()
            # 事件次数字段
            fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} AND event_type='{event}' THEN 1 ELSE 0 END) AS sp_express_event_{event_lower}_times_{win}d""")
            # 事件平台数字段
            fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} AND event_type='{event}' THEN sign ELSE NULL END) AS sp_express_event_{event_lower}_plat_cnt_{win}d""")

    # ========== 3. 不同事件行为最近一次与最远一次距今天数 ==========
    time_cond = time_cond_template.format(time_col="event_time", win=win)
    for event in event_arr:
        event_lower = event.lower()
        # 单个事件最近天数
        fields.append(f"""
        datediff(back_date, MAX(CASE WHEN {time_cond} AND event_type='{event}' THEN event_time ELSE NULL END)) AS sp_express_event_{event_lower}_recent_dura_d_beforep_{win}d""")
        # 单个事件最远天数
        fields.append(f"""
        datediff(back_date, MIN(CASE WHEN {time_cond} AND event_type='{event}' THEN event_time ELSE NULL END)) AS sp_express_event_{event_lower}_remote_dura_d_beforep_{win}d""")

    # ========== 4. 不同事件占总事件的比率（百分比） ==========
    for win in win_days:
        for event in event_arr:
            event_lower = event.lower()
            fields.append(f"""
            ROUND(CASE WHEN sp_express_event_total_times_{win}d = 0 THEN 0 ELSE sp_express_event_{event_lower}_times_{win}d / sp_express_event_total_times_{win}d END, 6) AS sp_express_event_{event_lower}_ratio_total_{win}d""")
    # 拼接所有字段，去除多余换行和空格，保持格式整洁
    return ",\n".join([f.strip() for f in fields])

def generate_express_stat_fields_levelv2(event_arr):
    fields = []
    # ========== 5. 不同事件 不同时间窗口占比（小窗口/大窗口） ==========
    win_mapping = {
        15: 30,   # 15天/30天
        30: 90,   # 30天/90天
        60: 360,  # 60天/360天
    }
    for small_win in [15, 30, 60]:
        big_win = win_mapping[small_win]
        # 事件维度：单个事件小窗口次数 / 该事件大窗口次数（保留6位小数）
        for event in event_arr:
            event_lower = event.lower()
            fields.append(f"""
            ROUND(CASE WHEN sp_express_event_{event_lower}_times_{big_win}d = 0 THEN 0 ELSE sp_express_event_{event_lower}_times_{small_win}d / sp_express_event_{event_lower}_times_{big_win}d END, 6) AS sp_express_event_{event_lower}_ratio_{small_win}d_vs_{big_win}d""")
    return ",\n".join([f.strip() for f in fields])


# ===================== express模块配置项 =====================
WIN_DAYS = [15, 30, 90, 180, 360]
EVENT_ARR = ["A01", "A02", "A03", "A04"]

# ===================== 生成最终SQL =====================
express_stat_fields = generate_express_stat_fields(WIN_DAYS, EVENT_ARR)

express_stat_fields_levelv2 = generate_express_stat_fields_levelv2(EVENT_ARR)
# 拼接最终INSERT SQL
aggregated_sql = f"""{base_sql},aggregated_data AS (
    SELECT
        mobile_md5,
        back_date,
        {express_stat_fields}
    FROM customer_dwb_express
    GROUP BY mobile_md5, back_date
)
INSERT OVERWRITE TABLE sms_bd_data.customer_test_offline_express_feature_2026010901
SELECT
    *,
    {express_stat_fields_levelv2}
FROM aggregated_data
"""
# 打印验证（查看生成的express模块特征）
print(aggregated_sql)

WITH customer_dwb_express AS (SELECT * FROM sms_bd_data.customer_dwb_express) ,aggregated_data AS (
    SELECT
        mobile_md5,
        back_date,
        SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, 15) THEN 1 ELSE 0 END) AS sp_express_event_total_times_15d,
COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, 15) THEN sign ELSE NULL END) AS sp_express_plat_cnt_15d,
SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, 30) THEN 1 ELSE 0 END) AS sp_express_event_total_times_30d,
COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, 30) THEN sign ELSE NULL END) AS sp_express_plat_cnt_30d,
SUM(CASE WHEN event_time < back_date AND event_time >= date_sub(back_date, 90) THEN 1 ELSE 0 END) AS sp_express_event_total_times_90d,
COUNT(DISTINCT CASE WHEN the_date < back_date AND the_date >= date_sub(back_date, 90) THEN sign ELSE NULL END) AS sp_express_plat_cnt_90d,
SUM(CASE WHEN event_

### 金融信贷特征衍生代码及字段含义

In [52]:
def generate_fin_loan_stat_fields(win_days, ind_arr, event_arr, field_meta):
    """
    封装统计字段生成逻辑，返回格式化的字段字符串，同时填充字段元数据
    :param win_days: 时间窗口列表
    :param ind_arr: 行业标签列表
    :param event_arr: 事件类型列表
    :param field_meta: 字段元数据字典（用于存储字段名、类型、说明）
    :return: 拼接好的统计字段字符串
    """
    fields = []
    # 统一时间条件模板（减少重复书写）
    time_cond_template = "{time_col} < back_date AND {time_col} >= date_sub(back_date, {win})"
    
    # ========== 1. 金融事件总次数 ==========
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="event_time")
        
        # 总次数字段
        total_times_field = f"sp_fin_loan_total_times_{win}d"
        fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} THEN 1 ELSE 0 END) AS {total_times_field}""")
        # 填充字段元数据
        field_meta[total_times_field] = {
            "type": "BIGINT",
            "cn_desc": f"近{win}天内金融事件总次数",
            "biz_tag": "总次数"
        }
        
        # 平台数字段
        plat_cnt_field = f"sp_fin_loan_plat_cnt_{win}d"
        fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} THEN sign ELSE NULL END) AS {plat_cnt_field}""")
        field_meta[plat_cnt_field] = {
            "type": "INT",
            "cn_desc": f"近{win}天内金融事件平台数",
            "biz_tag": "平台数"
        }

    # ========== 2. 不同行业行为次数 ==========
    # 行业中文注释映射
    ind_cn_mapping = {
        "indA": "A类小贷",
        "indB": "B类小贷",
        "indC": "C类小贷",
        "indD": "三方催收",
        "indE": "消费金融",
        "indF": "银行",
        "indG": "其他机构"
    }
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="event_time")
        for ind in ind_arr:
            ind_lower = ind.lower()
            ind_cn = ind_cn_mapping.get(ind, ind)
            
            # 行业次数字段
            ind_times_field = f"sp_fin_loan_{ind_lower}_times_{win}d"
            fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} AND ind_tag='{ind}' THEN 1 ELSE 0 END) AS {ind_times_field}""")
            field_meta[ind_times_field] = {
                "type": "BIGINT",
                "cn_desc": f"近{win}天内{ind_cn}行业金融事件次数",
                "biz_tag": f"行业次数|{ind_cn}"
            }
            
            # 行业平台数字段
            ind_plat_field = f"sp_fin_loan_{ind_lower}_plat_cnt_{win}d"
            fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} AND ind_tag='{ind}' THEN sign ELSE NULL END) AS {ind_plat_field}""")
            field_meta[ind_plat_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{ind_cn}行业金融事件平台数",
                "biz_tag": f"行业平台数|{ind_cn}"
            }

    # ========== 3. 不同事件类型行为次数 ==========
    # 事件中文注释映射
    event_cn_mapping = {
        "B01": "严重逾期",
        "B02": "三方催收",
        "B03": "中度逾期",
        "B04": "还款失败",
        "B05": "申请失败",
        "B06": "还款成功",
        "B07": "放款成功",
        "B08": "还款提醒",
        "B09": "贷款营销",
        "B10": "金融其他"
    }
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="event_time")
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            
            # 事件次数字段
            event_times_field = f"sp_fin_loan_event_{event_lower}_times_{win}d"
            fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} AND event_type='{event}' THEN 1 ELSE 0 END) AS {event_times_field}""")
            field_meta[event_times_field] = {
                "type": "BIGINT",
                "cn_desc": f"近{win}天内{event_cn}事件次数",
                "biz_tag": f"事件次数|{event_cn}"
            }
            
            # 事件平台数字段
            event_plat_field = f"sp_fin_loan_event_{event_lower}_plat_cnt_{win}d"
            fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} AND event_type='{event}' THEN sign ELSE NULL END) AS {event_plat_field}""")
            field_meta[event_plat_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{event_cn}事件平台数",
                "biz_tag": f"事件平台数|{event_cn}"
            }

    # ========== 4. 行业+事件类型组合次数 ==========
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="event_time")
        for ind in ind_arr:
            ind_lower = ind.lower()
            if ind_lower == "indd":
                continue
            ind_cn = ind_cn_mapping.get(ind, ind)
            
            for event in event_arr:
                event_lower = event.lower()
                event_cn = event_cn_mapping.get(event, event)
                
                # 组合次数字段
                combo_times_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_times_{win}d"
                fields.append(f"""SUM(CASE WHEN {sum_time_cond} AND ind_tag='{ind}' AND event_type='{event}' THEN 1 ELSE 0 END) AS {combo_times_field}""")
                field_meta[combo_times_field] = {
                    "type": "BIGINT",
                    "cn_desc": f"近{win}天内{ind_cn}行业{event_cn}事件次数",
                    "biz_tag": f"行业+事件次数|{ind_cn}|{event_cn}"
                }
                
                # 组合平台数字段
                combo_plat_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_plat_cnt_{win}d"
                fields.append(f"""COUNT(DISTINCT CASE WHEN {count_time_cond} AND ind_tag='{ind}' AND event_type='{event}' THEN sign ELSE NULL END) AS {combo_plat_field}""")
                field_meta[combo_plat_field] = {
                    "type": "INT",
                    "cn_desc": f"近{win}天内{ind_cn}行业{event_cn}事件平台数",
                    "biz_tag": f"行业+事件平台数|{ind_cn}|{event_cn}"
                }

    # ========== 5. 不同事件行为最近一次与最远一次距今天数 ==========
    time_cond = time_cond_template.format(time_col="event_time", win=win)
    for event in event_arr:
        event_lower = event.lower()
        event_cn = event_cn_mapping.get(event, event)
        
        # 单个事件最近天数
        event_latest_field = f"sp_fin_loan_event_{event_lower}_recent_dura_d_beforep_{win}d"
        fields.append(f"""datediff(back_date, MAX(CASE WHEN {time_cond} AND event_type='{event}' THEN event_time ELSE NULL END)) AS {event_latest_field}""")
        field_meta[event_latest_field] = {
            "type": "INT",
            "cn_desc": f"近{win}天内{event_cn}事件最近一次距今天数",
            "biz_tag": f"事件天数|{event_cn}|最近"
        }
        
        # 单个事件最远天数
        event_farthest_field = f"sp_fin_loan_event_{event_lower}_remote_dura_d_beforep_{win}d"
        fields.append(f"""datediff(back_date, MIN(CASE WHEN {time_cond} AND event_type='{event}' THEN event_time ELSE NULL END)) AS {event_farthest_field}""")
        field_meta[event_farthest_field] = {
            "type": "INT",
            "cn_desc": f"近{win}天内{event_cn}事件最远一次距今天数",
            "biz_tag": f"事件天数|{event_cn}|最远"
        }
    
    for ind in ind_arr:
        ind_lower = ind.lower()
        if ind_lower == "indd":
            continue
        ind_cn = ind_cn_mapping.get(ind, ind)
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            
            # 组合事件最近天数
            combo_latest_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_recent_dura_d_beforep_{win}d"
            fields.append(f"""datediff(back_date, MAX(CASE WHEN {time_cond} AND ind_tag='{ind}' AND event_type='{event}' THEN event_time ELSE NULL END)) AS {combo_latest_field}""")
            field_meta[combo_latest_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{ind_cn}行业{event_cn}事件最近一次距今天数",
                "biz_tag": f"行业+事件天数|{ind_cn}|{event_cn}|最近"
            }
            
            # 组合事件最远天数
            combo_farthest_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_remote_dura_d_beforep_{win}d"
            fields.append(f"""datediff(back_date, MIN(CASE WHEN {time_cond} AND ind_tag='{ind}' AND event_type='{event}' THEN event_time ELSE NULL END)) AS {combo_farthest_field}""")
            field_meta[combo_farthest_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{ind_cn}行业{event_cn}事件最远一次距今天数",
                "biz_tag": f"行业+事件天数|{ind_cn}|{event_cn}|最远"
            }
    # 拼接所有字段，去除多余换行和空格
    return ",\n".join([f.strip() for f in fields])

def generate_fin_loan_stat_fields_levelv2(win_days, event_arr, ind_arr, field_meta):
    """
    生成比率类字段，同时填充字段元数据
    """
    fields = []
    # 事件/行业中文映射（复用）
    event_cn_mapping = {
        "B01": "严重逾期",
        "B02": "三方催收",
        "B03": "中度逾期",
        "B04": "还款失败",
        "B05": "申请失败",
        "B06": "还款成功",
        "B07": "放款成功",
        "B08": "还款提醒",
        "B09": "贷款营销",
        "B10": "金融其他"
    }
    ind_cn_mapping = {
        "indA": "A类小贷",
        "indB": "B类小贷",
        "indC": "C类小贷",
        "indD": "三方催收",
        "indE": "消费金融",
        "indF": "银行",
        "indG": "其他机构"
    }
    
    # ========== 6. 不同事件/行业占总事件比率 ==========
    for win in win_days:
        # 6.1 单个事件占总事件的比率
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            event_ratio_field = f"sp_fin_loan_event_{event_lower}_ratio_total_{win}d"
            fields.append(f"""
            ROUND(CASE WHEN sp_fin_loan_total_times_{win}d = 0 THEN 0 ELSE sp_fin_loan_event_{event_lower}_times_{win}d / sp_fin_loan_total_times_{win}d END, 6) AS {event_ratio_field}""")
            field_meta[event_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{win}天内{event_cn}事件次数占总事件次数的比率（保留6位小数）",
                "biz_tag": f"事件占比|{event_cn}"
            }
        
        # 6.2 单个行业占总事件的比率
        for ind in ind_arr:
            ind_lower = ind.lower()
            ind_cn = ind_cn_mapping.get(ind, ind)
            ind_ratio_field = f"sp_fin_loan_{ind_lower}_ratio_total_{win}d"
            fields.append(f"""
            ROUND(CASE WHEN sp_fin_loan_total_times_{win}d = 0 THEN 0 ELSE sp_fin_loan_{ind_lower}_times_{win}d / sp_fin_loan_total_times_{win}d END, 6) AS {ind_ratio_field}""")
            field_meta[ind_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{win}天内{ind_cn}行业事件次数占总事件次数的比率（保留6位小数）",
                "biz_tag": f"行业占比|{ind_cn}"
            }

    # ========== 7. 不同事件/行业+事件跨窗口占比 ==========
    win_mapping = {
        15: 30,   # 15天/30天
        30: 90,   # 30天/90天
        60: 360   # 60天/360天
    }
    for small_win in [15, 30, 60]:
        big_win = win_mapping[small_win]
        
        # 7.1 纯事件维度跨窗口占比
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            event_win_ratio_field = f"sp_fin_loan_event_{event_lower}_ratio_{small_win}to{big_win}d"
            fields.append(f"""ROUND(CASE WHEN sp_fin_loan_event_{event_lower}_times_{big_win}d = 0 THEN 0 ELSE sp_fin_loan_event_{event_lower}_times_{small_win}d / sp_fin_loan_event_{event_lower}_times_{big_win}d END, 6) AS {event_win_ratio_field}""")
            field_meta[event_win_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{small_win}天内{event_cn}事件次数占近{big_win}天次数的比率（保留6位小数）",
                "biz_tag": f"事件跨窗口占比|{event_cn}|{small_win}d/{big_win}d"
            }
        
        # 7.2 行业+事件组合维度跨窗口占比
        for ind in ind_arr:
            ind_lower = ind.lower()
            if ind_lower == "indd":
                continue
            ind_cn = ind_cn_mapping.get(ind, ind)
            for event in event_arr:
                event_lower = event.lower()
                event_cn = event_cn_mapping.get(event, event)
                combo_win_ratio_field = f"sp_fin_loan_{ind_lower}_event_{event_lower}_ratio_{small_win}to{big_win}d"
                fields.append(f"""ROUND(CASE WHEN sp_fin_loan_{ind_lower}_event_{event_lower}_times_{big_win}d = 0 THEN 0 ELSE sp_fin_loan_{ind_lower}_event_{event_lower}_times_{small_win}d / sp_fin_loan_{ind_lower}_event_{event_lower}_times_{big_win}d END, 6) AS {combo_win_ratio_field}""")
                field_meta[combo_win_ratio_field] = {
                    "type": "DOUBLE",
                    "cn_desc": f"近{small_win}天内{ind_cn}行业{event_cn}事件次数占近{big_win}天次数的比率（保留6位小数）",
                    "biz_tag": f"行业+事件跨窗口占比|{ind_cn}|{event_cn}|{small_win}d/{big_win}d"
                }
    
    return ",\n".join([f.strip() for f in fields])

# ===================== 主流程 =====================
# 初始化字段元数据字典
field_mapping_dict = {
    # 基础字段（非统计字段）
    "mobile_md5": {"type": "STRING", "cn_desc": "手机号MD5加密值", "biz_tag": "基础标识"},
    "back_date": {"type": "DATE", "cn_desc": "回溯日期", "biz_tag": "基础标识"}
}

# 配置项
WIN_DAYS = [15, 30, 60, 90, 180, 360]
IND_ARR = ["indA", "indB", "indC", "indD", "indE", "indF", "indG"]
EVENT_ARR = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B09", "B10"]

# 基础SQL（替换为实际值）
base_sql = """
CREATE TABLE sms_bd_data.customer_test_offline_express_feature_2026010901 AS
WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 需要满足回溯日期
        -- back_date要求YYYYMMDD格式
        select
            phone as mobile_md5,
            substr(submitdate, 0, 10) as back_date
        from sms_bd_data.ods_yxx_spn_sms_detail_di
        where submitdate>='2025-08-20'
        limit 50000
    )smp
    group by mobile_md5, back_date
),
customer_dwb_sms AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_time,
        event_type, 
        ind_tag
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_loan.sign,
            dwb_loan.event_type,
            dwb_loan.event_time,
            case when dwb_loan.sign regexp "抖音月付|放心借|微众银行|花呗" then 'indA'
                 when dwb_loan.sign regexp "美团月付|携程金融|360借|拍拍贷|分期乐|拍拍金融|360贷款|上海拍拍贷|美团金融服务" then 'indB'
                 when dwb_loan.sign regexp "还呗|信用飞钱包|省呗|乐享借|融360|洋钱罐借款|众安金融|分期金融|信用飞|乐逸花|极融借款|好分期|信用卡贷|金瀛分期|小赢卡贷|宜享花|众安贷借钱|融逸花|卡贷金融|吉用花|时光分期|小花钱包|借钱呗|金豆花|信用飞金融|来分期|小花借款|好会借|乐贷分期|及贷|榕树贷款|融易分期|借小花|薇钱包|你我贷|玖富借条" then 'indC'
                 when dwb_loan.sign regexp "金融调解|数科金融|满松科技|利信普惠|普信金融|卡卡金融|和信普惠|和信金融|利信金融|数信普惠|普惠快信|普惠信息|首山金融|普惠金融|数信普惠|玖富万卡|钱站|鹰潭市金融纠纷调解中心|国美易卡|普惠分期|上海金融|数科纠纷调解中心" or sign in ('玖富') then 'indD'
                 when dwb_loan.sign regexp "消费分期|消费金融|招联金融|移动白条|捷信金融|马上金融|中原消费金融|马上消费|分期消费" then 'indE'
                 when dwb_loan.sign regexp "银行|农信|农商" then 'indF'
            else 'indG' end as ind_tag
        from cus_sample cus_smp
        left join(
            select
                phone,
                sign,
                event_type,
                from_unixtime(unix_timestamp(the_date, 'yyyyMMdd'), 'yyyy-MM-dd') as event_time
            from sms_bd_data.sms_dwb_fin_loan_event_fdt
        )dwb_loan
        on cus_smp.mobile_md5=dwb_loan.phone 
        where cus_smp.back_date > dwb_loan.event_time and dwb_loan.event_time >= date_sub(cus_smp.back_date, 360)
    )cus_dwb
    group by mobile_md5, back_date, sign, event_type, event_time, ind_tag
)
"""

# 生成统计字段（v1 + v2）
stat_fields_v1 = generate_fin_loan_stat_fields(WIN_DAYS, IND_ARR, EVENT_ARR, field_mapping_dict)
stat_fields_v2 = generate_fin_loan_stat_fields_levelv2(WIN_DAYS, EVENT_ARR, IND_ARR, field_mapping_dict)

# 拼接最终SQL
aggregated_sql = f"""{base_sql},aggregated_data AS (
    SELECT
        mobile_md5,
        back_date,
        {stat_fields_v1}
    FROM customer_dwb_sms
    GROUP BY mobile_md5, back_date
)
-- INSERT OVERWRITE TABLE sms_bd_data.customer_test_offline_sp_feature_2026010901
SELECT
   *,
    {stat_fields_v2}
FROM aggregated_data
"""

# ===================== 输出结果 =====================
# 1. 打印生成的SQL
print("===== 生成的SQL =====")
print(aggregated_sql)  # 打印前2000字符，避免过长


===== 生成的SQL =====

CREATE TABLE sms_bd_data.customer_test_offline_express_feature_2026010901 AS
WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 需要满足回溯日期
        -- back_date要求YYYYMMDD格式
        select
            phone as mobile_md5,
            substr(submitdate, 0, 10) as back_date
        from sms_bd_data.ods_yxx_spn_sms_detail_di
        where submitdate>='2025-08-20'
        limit 50000
    )smp
    group by mobile_md5, back_date
),
customer_dwb_sms AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_time,
        event_type, 
        ind_tag
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_loan.sign,
            dwb_loan.event_type,
            dwb_loan.event_time,
            case when dwb_loan.sign regexp "抖音月付|放心借|微众银行|花呗" then 'indA'
                 when dwb_loan.sign regexp "美团月付|携程金融|360借|拍拍贷|分期乐|拍拍金融|360贷款|上海拍拍贷|美团金融服务" then 'indB'
        

In [53]:
# 2. 打印字段映射字典（格式化输出）
print("\n===== 字段含义与类型映射字典 =====")
import json
print(json.dumps(field_mapping_dict, ensure_ascii=False, indent=4))
field_mapping_dataframe = pd.DataFrame(field_mapping_dict).T

# 3. 可选：输出字段类型统计
print("\n===== 字段类型统计 =====")
type_count = {}
for field, meta in field_mapping_dict.items():
    field_type = meta["type"]
    type_count[field_type] = type_count.get(field_type, 0) + 1
for t, cnt in type_count.items():
    print(f"{t}类型字段数：{cnt}")


===== 字段含义与类型映射字典 =====
{
    "mobile_md5": {
        "type": "STRING",
        "cn_desc": "手机号MD5加密值",
        "biz_tag": "基础标识"
    },
    "back_date": {
        "type": "DATE",
        "cn_desc": "回溯日期",
        "biz_tag": "基础标识"
    },
    "sp_fin_loan_total_times_15d": {
        "type": "BIGINT",
        "cn_desc": "近15天内金融事件总次数",
        "biz_tag": "总次数"
    },
    "sp_fin_loan_plat_cnt_15d": {
        "type": "INT",
        "cn_desc": "近15天内金融事件平台数",
        "biz_tag": "平台数"
    },
    "sp_fin_loan_total_times_30d": {
        "type": "BIGINT",
        "cn_desc": "近30天内金融事件总次数",
        "biz_tag": "总次数"
    },
    "sp_fin_loan_plat_cnt_30d": {
        "type": "INT",
        "cn_desc": "近30天内金融事件平台数",
        "biz_tag": "平台数"
    },
    "sp_fin_loan_total_times_60d": {
        "type": "BIGINT",
        "cn_desc": "近60天内金融事件总次数",
        "biz_tag": "总次数"
    },
    "sp_fin_loan_plat_cnt_60d": {
        "type": "INT",
        "cn_desc": "近60天内金融事件平台数",
        "biz_tag": "平台数"
    

In [54]:
field_mapping_dataframe = pd.DataFrame(field_mapping_dict).T
field_mapping_dataframe.to_excel("fin_loan_field_mapping_output.xlsx", index_label="field_name")

### 快递特征衍生代码及字段含义

In [ ]:
def generate_express_stat_fields(win_days, event_arr, field_meta):
    """
    封装快递（express）模块特征生成逻辑（仅事件维度，无行业分组）
    同步fin_loan优化逻辑：新增字段元数据、中文注释、类型标注
    :param win_days: 时间窗口列表
    :param event_arr: 事件类型列表
    :param field_meta: 字段元数据字典（用于存储字段名、类型、说明）
    :return: 拼接好的统计字段字符串
    """
    fields = []
    # 统一时间条件模板
    time_cond_template = "{time_col} < back_date AND {time_col} >= date_sub(back_date, {win})"
    
    # ========== 1. 快递事件总次数 + 平台数 ==========
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="event_time")
        
        # 总次数字段
        total_times_field = f"sp_express_total_times_{win}d"
        fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} THEN 1 ELSE 0 END) AS {total_times_field}""")
        # 填充字段元数据
        field_meta[total_times_field] = {
            "type": "BIGINT",
            "cn_desc": f"近{win}天内快递事件总次数",
            "biz_tag": "总次数"
        }
        
        # 平台数字段
        plat_cnt_field = f"sp_express_plat_cnt_{win}d"
        fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} THEN sign ELSE NULL END) AS {plat_cnt_field}""")
        field_meta[plat_cnt_field] = {
            "type": "INT",
            "cn_desc": f"近{win}天内快递事件平台数",
            "biz_tag": "平台数"
        }

    # ========== 2. 不同事件类型行为次数 + 平台数 ==========
    # 快递事件中文注释映射（可根据实际业务补充）
    event_cn_mapping = {
        "A01": "快递签收",
        "A02": "快递投诉",
        "A03": "快递寄件",
        "A04": "其他快递"
    }
    for win in win_days:
        sum_time_cond = time_cond_template.format(win=win, time_col="event_time")
        count_time_cond = time_cond_template.format(win=win, time_col="event_time")
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            
            # 事件次数字段
            event_times_field = f"sp_express_event_{event_lower}_times_{win}d"
            fields.append(f"""
            SUM(CASE WHEN {sum_time_cond} AND event_type='{event}' THEN 1 ELSE 0 END) AS {event_times_field}""")
            field_meta[event_times_field] = {
                "type": "BIGINT",
                "cn_desc": f"近{win}天内{event_cn}事件次数",
                "biz_tag": f"事件次数|{event_cn}"
            }
            
            # 事件平台数字段
            event_plat_field = f"sp_express_event_{event_lower}_plat_cnt_{win}d"
            fields.append(f"""
            COUNT(DISTINCT CASE WHEN {count_time_cond} AND event_type='{event}' THEN sign ELSE NULL END) AS {event_plat_field}""")
            field_meta[event_plat_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{event_cn}事件涉及的平台数",
                "biz_tag": f"事件平台数|{event_cn}"
            }

    # ========== 3. 不同事件行为最近一次与最远一次距今天数 ==========
    for win in win_days:  # 修复win循环漏写问题
        time_cond = time_cond_template.format(time_col="event_time", win=win)
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            
            # 单个事件最近天数
            event_latest_field = f"sp_express_event_{event_lower}_recent_dura_d_beforep_{win}d"
            fields.append(f"""
            datediff(back_date, MAX(CASE WHEN {time_cond} AND event_type='{event}' THEN event_time ELSE NULL END)) AS {event_latest_field}""")
            field_meta[event_latest_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{event_cn}事件最近一次距回溯日期的天数",
                "biz_tag": f"事件天数|{event_cn}|最近"
            }
            
            # 单个事件最远天数
            event_farthest_field = f"sp_express_event_{event_lower}_remote_dura_d_beforep_{win}d"
            fields.append(f"""
            datediff(back_date, MIN(CASE WHEN {time_cond} AND event_type='{event}' THEN event_time ELSE NULL END)) AS {event_farthest_field}""")
            field_meta[event_farthest_field] = {
                "type": "INT",
                "cn_desc": f"近{win}天内{event_cn}事件最远一次距回溯日期的天数",
                "biz_tag": f"事件天数|{event_cn}|最远"
            }

    # 拼接所有字段，去除多余换行和空格
    return ",\n".join([f.strip() for f in fields])

def generate_express_stat_fields_levelv2(win_days, event_arr, field_meta):
    """
    生成快递模块比率类字段，同步fin_loan优化逻辑
    """
    fields = []
    # 快递事件中文注释映射（复用）
    event_cn_mapping = {
        "A01": "快递签收",
        "A02": "快递揽收",
        "A03": "快递派送",
        "A04": "快递异常"
    }
    
    # ========== 4. 不同事件占总事件的比率 ==========
    for win in win_days:
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            event_ratio_field = f"sp_express_event_{event_lower}_ratio_total_{win}d"
            fields.append(f"""
            ROUND(CASE WHEN sp_express_total_times_{win}d = 0 THEN 0 ELSE sp_express_event_{event_lower}_times_{win}d / sp_express_total_times_{win}d END, 6) AS {event_ratio_field}""")
            field_meta[event_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{win}天内{event_cn}事件次数占总事件次数的比率（保留6位小数）",
                "biz_tag": f"事件占比|{event_cn}"
            }

    # ========== 5. 不同事件跨窗口占比（小窗口/大窗口） ==========
    # 自定义窗口映射（与fin_loan保持一致）
    win_mapping = {
        15: 30,   # 15天/30天
        30: 90,   # 30天/90天
        90: 180,  # 90天/180天
        180: 360, # 180天/360天
        360: 360  # 360天自身
    }
    for small_win in win_days:
        big_win = win_mapping[small_win]
        # 仅处理存在的大窗口（避免字段不存在）
        if big_win not in win_days:
            continue
        
        # 纯事件维度跨窗口占比
        for event in event_arr:
            event_lower = event.lower()
            event_cn = event_cn_mapping.get(event, event)
            event_win_ratio_field = f"sp_express_event_{event_lower}_ratio_{small_win}to{big_win}d"
            fields.append(f"""
            ROUND(CASE WHEN sp_express_event_{event_lower}_times_{big_win}d = 0 THEN 0 ELSE sp_express_event_{event_lower}_times_{small_win}d / sp_express_event_{event_lower}_times_{big_win}d END, 6) AS {event_win_ratio_field}""")
            field_meta[event_win_ratio_field] = {
                "type": "DOUBLE",
                "cn_desc": f"近{small_win}天内{event_cn}事件次数占近{big_win}天次数的比率（保留6位小数）",
                "biz_tag": f"事件跨窗口占比|{event_cn}|{small_win}d/{big_win}d"
            }
    
    return ",\n".join([f.strip() for f in fields])

# ===================== 主流程 =====================
# 初始化字段元数据字典
field_mapping_dict = {
    # 基础字段（非统计字段）
    "mobile_md5": {"type": "STRING", "cn_desc": "手机号MD5加密值", "biz_tag": "基础标识"},
    "back_date": {"type": "DATE", "cn_desc": "回溯日期", "biz_tag": "基础标识"}
}

# express模块配置项（与需求一致）
WIN_DAYS = [15, 30, 90, 180, 360]
EVENT_ARR = ["A01", "A02", "A03", "A04"]

# 基础SQL（参考fin_loan结构，适配快递业务源表）
base_sql = """
CREATE TABLE sms_bd_data.customer_test_offline_express_feature_2026010901 AS
WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 快递业务样本表（示例结构）
        select 
            mobile_md5,
            substr(back_date, 0, 10) as back_date
        from risk.express_intent_samples_20241023
    )smp
    group by mobile_md5, back_date
),
customer_dwb_express AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_type,
        event_time
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_express.sign,
            dwb_express.event_type,
            dwb_express.event_time
        from cus_sample cus_smp
        left join(
            select
                phone,
                sign,
                event_type,
                from_unixtime(unix_timestamp(the_date, 'yyyyMMdd'), 'yyyy-MM-dd') as event_time
            from sms_bd_data.sms_dwb_express_event_fdt
        )dwb_express
        on cus_smp.mobile_md5=dwb_express.phone 
        where cus_smp.back_date > dwb_express.event_time and dwb_express.event_time >= date_sub(cus_smp.back_date, 360)
    )cus_dwb
    group by mobile_md5, back_date, sign, event_type, event_time
)
"""

# 生成统计字段（v1 + v2）
exp_stat_fields_v1 = generate_express_stat_fields(WIN_DAYS, EVENT_ARR, field_mapping_dict)
exp_stat_fields_v2 = generate_express_stat_fields_levelv2(WIN_DAYS, EVENT_ARR, field_mapping_dict)

# 拼接最终SQL
exp_aggregated_sql = f"""{base_sql},aggregated_data AS (
    SELECT
        mobile_md5,
        back_date,
        {exp_stat_fields_v1}
    FROM customer_dwb_express
    GROUP BY mobile_md5, back_date
)
-- INSERT OVERWRITE TABLE sms_bd_data.customer_test_offline_express_feature_2026010901
SELECT
   *,
    {exp_stat_fields_v2}
FROM aggregated_data
"""

# ===================== 输出结果 =====================
# 1. 打印生成的SQL（截取前2000字符，避免过长）
print("===== 生成的快递模块SQL =====")
print(exp_aggregated_sql)


===== 生成的快递模块SQL =====

WITH cus_sample AS (
    SELECT 
        mobile_md5,
        back_date
    FROM(
        -- 快递业务样本表（示例结构）
        select 
            mobile_md5,
            substr(back_date, 0, 10) as back_date
        from risk.express_intent_samples_20241023
    )smp
    group by mobile_md5, back_date
),
customer_dwb_express AS (
    select
        mobile_md5, 
        back_date, 
        sign, 
        event_type,
        event_time  -- 统一时间字段命名
    from(
        select
            cus_smp.mobile_md5,
            cus_smp.back_date,
            dwb_express.sign,
            dwb_express.event_type,
            dwb_express.event_time
        from cus_sample cus_smp
        left join(
            select
                phone,
                sign,
                event_type,
                from_unixtime(unix_timestamp(the_date, 'yyyyMMdd'), 'yyyy-MM-dd') as event_time
            from sms_bd_data.sms_dwb_express_event_fdt
        )dwb_express
        on cus_smp.mobile_md5=dwb_

In [41]:
# 2. 打印字段映射字典（格式化输出）
print("\n===== Express模块字段含义与类型映射字典 =====")
import json
print(json.dumps(field_mapping_dict, ensure_ascii=False, indent=4))


# 3. 输出字段类型统计
print("\n===== Express模块字段类型统计 =====")
type_count = {}
for field, meta in field_mapping_dict.items():
    field_type = meta["type"]
    type_count[field_type] = type_count.get(field_type, 0) + 1
for t, cnt in type_count.items():
    print(f"{t}类型字段数：{cnt}")


===== Express模块字段含义与类型映射字典 =====
{
    "mobile_md5": {
        "type": "STRING",
        "cn_desc": "手机号MD5加密值",
        "biz_tag": "基础标识"
    },
    "back_date": {
        "type": "DATE",
        "cn_desc": "回溯日期",
        "biz_tag": "基础标识"
    },
    "sp_express_total_times_15d": {
        "type": "BIGINT",
        "cn_desc": "近15天内快递事件总次数",
        "biz_tag": "总次数"
    },
    "sp_express_plat_cnt_15d": {
        "type": "INT",
        "cn_desc": "近15天内快递事件平台数",
        "biz_tag": "平台数"
    },
    "sp_express_total_times_30d": {
        "type": "BIGINT",
        "cn_desc": "近30天内快递事件总次数",
        "biz_tag": "总次数"
    },
    "sp_express_plat_cnt_30d": {
        "type": "INT",
        "cn_desc": "近30天内快递事件平台数",
        "biz_tag": "平台数"
    },
    "sp_express_total_times_90d": {
        "type": "BIGINT",
        "cn_desc": "近90天内快递事件总次数",
        "biz_tag": "总次数"
    },
    "sp_express_plat_cnt_90d": {
        "type": "INT",
        "cn_desc": "近90天内快递事件平台数",
        "biz_tag": "平台数"
 

In [42]:
exp_field_mapping_dataframe = pd.DataFrame(field_mapping_dict).T
exp_field_mapping_dataframe.to_excel("exp_field_mapping_output.xlsx", index_label="field_name")